In [1]:
import socket
import socketserver, time
from threading import Thread
from time import sleep

In [2]:
host = '25.93.32.9'  #http://25.94.218.230:555
port = 1234

timeout = 180
buffer_size = 1024

CONFIRMATION = bytes('Data processed', 'utf-8')
NO_DATA = bytes('no_data', 'utf-8')

# broadcast = ('<broadcast>', 1234)

messages = []

In [3]:
def control(x, in_min, in_max, out_min, out_max):
    return int((x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min)

In [4]:
def setup():
    server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
    
    server.settimeout(timeout)
    server.bind((host, port))

    print('Server rodando')
    
    return server

In [5]:
def send_request(server):
    
    request = bytes('r', 'utf-8')
    server.sendto(request, broadcast)
    
    print('Request sent')

In [6]:
def receive_msg(server):

    bytes_received, address = server.recvfrom(1024)
    
    message = bytes_received.decode()
    
#     print('bytes_received ', bytes_received)
    print('message: ', message)
    print('from: ', address)
        
    if message[0] == 'S':

        byte_value = bytes(message[1:], 'utf-8')
        
        bytes_to_send = int.from_bytes(byte_value, 'little')
        
        messages.append(bytes_to_send)
            
        server.sendto(CONFIRMATION, address)
        
    elif message[0] == 'A':
        
        if len(messages) != 0:
            
            to_send = messages[0]
            messages.pop(0)

            mv = control(to_send, 0, 1024, 100, 0)
            
            bytes_to_send = int.to_bytes(mv, 4, byteorder='little')
            
            server.sendto(bytes_to_send, address)
        else:
            server.sendto(NO_DATA, address)
            
    else:
        print('Cant interpretate request')

In [7]:
server = setup()

while True:
    receive_msg(server)
    time.sleep(0.25)

message:  Sc                               
from:  ('25.68.213.125', 63864)
message:  Sb                               
from:  ('25.68.213.125', 63864)
message:  Sb                               
from:  ('25.68.213.125', 63864)
message:  Sd                               
from:  ('25.68.213.125', 58301)
message:  Sd                               
from:  ('25.68.213.125', 58301)
message:  Sd                               
from:  ('25.68.213.125', 58301)
message:  Sd                               
from:  ('25.68.213.125', 58301)
message:  Sd                               
from:  ('25.68.213.125', 58301)
message:  Sd                               
from:  ('25.68.213.125', 58301)
message:  Sd                               
from:  ('25.68.213.125', 58301)
message:  Sb                               
from:  ('25.68.213.125', 58301)
message:  Sb                               
from:  ('25.68.213.125', 58301)


timeout: timed out

In [ ]:
# int.to_bytes(5,4,byteorder='little')
# a = int.to_bytes(5, 4, byteorder = 'little')
# s = b's' + a
# int.from_bytes(a, 'little')